In [2]:
import yt
import yt.analysis_modules.halo_analysis.api as haa
import yaml
import run_workflow as rw
import subprocess
import shutil
import glob
import os

In [3]:
seed_ids = ["%.2d" % i for i in range(1,11)]

workdir = '/dpool/cwagner4/working'
enzo_dir = '%s/enzo_sims' %workdir
enzo_par = '%s/parameter_file.txt' %enzo_dir
enzo_ex = '%s/enzo.exe' %enzo_dir
enzo_par_templ = '%s/parameter_file_template.txt' %enzo_dir
music_dir = '%s/music' %workdir
music_ex = '%s/MUSIC' %music_dir

In [11]:
seed_ids

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

In [12]:
os.chdir(music_dir)


for i in seed_ids:
    seed_idx=i
    seed = '%s/topostellar-oceans/resources/SEED%s' %(workdir,seed_idx)

    with open(seed, "r") as f:
        seed_val = f.readline()
        f.close()

    seed_dict = {'seed': seed_val, 'seed_id': 'SEED%s' %seed_idx}
    print(seed_dict)

    music_par = '%s/ics_template.conf' %music_dir
    music_ic = '%s/SEED%s' %(music_dir,seed_idx)

    with open(music_par, "r") as f:
        contents = f.read()
        f.close()

    with open('%s/output_ics_SEED%s.conf' %(music_dir,seed_idx), "a+") as f:
        f.write(contents.format(**seed_dict))
        f.close()

{'seed': '21188\n', 'seed_id': 'SEED01'}
{'seed': '55488\n', 'seed_id': 'SEED02'}
{'seed': '45027\n', 'seed_id': 'SEED03'}
{'seed': '44292\n', 'seed_id': 'SEED04'}
{'seed': '16270\n', 'seed_id': 'SEED05'}
{'seed': '48438\n', 'seed_id': 'SEED06'}
{'seed': '13127\n', 'seed_id': 'SEED07'}
{'seed': '15915\n', 'seed_id': 'SEED08'}
{'seed': '95322\n', 'seed_id': 'SEED09'}
{'seed': '96840\n', 'seed_id': 'SEED10'}


In [17]:
for i in seed_ids[:]:
    seed_idx = i

    enzo_results = '%s/output/seeds/SEED%s' %(enzo_dir, seed_idx)
    halo_coords = '%s/coords.v' %enzo_results

    with open(halo_coords) as c:
        crds = yaml.load(c, Loader=yaml.FullLoader)
    seed = '%s/topostellar-oceans/resources/SEED%s' %(workdir,seed_idx) 

    with open(seed, "r") as f:
        seed_val = f.readline()
        f.close()
    
    seed_dict = {'seed': seed_val, 'seed_id': 'SEED%s' %seed_idx}
    # print(seed_dict)

    else:
        center_dict = {'center_1': crds[0], 'center_2': crds[1], 'center_3':  crds[2]}
        center_dict.update(seed_dict)
        
        music_par = '%s/ics_centered_template.conf' %music_dir
        music_ic = '%s/SEED%s_initial_centered' %(music_dir,seed_idx)

        with open(music_par, "r") as f:
            contents = f.read()
            f.close()
        
        with open('%s/output_ics_centered_%s.conf' %(music_dir,seed_idx), "a+") as f:
            f.write(contents.format(**center_dict))
            f.close()

SyntaxError: invalid syntax (<ipython-input-17-4b99a5f98ab4>, line 18)

In [12]:
### Generating initial conditions
os.chdir(music_dir)

for i in seed_ids[1:2]:
    seed_idx = i
    refine = False

    seed = '%s/topostellar-oceans/resources/SEED%s' %(workdir,seed_idx) 

    rw.generate_ic(seed=seed,music_dir=music_dir,
                seed_idx=seed_idx,enzo_dir=enzo_dir,
                music_ex=music_ex,refine=refine)

{'seed': '55488\n', 'seed_id': 'SEED02'}
Generated first set of IC


In [13]:
### Run enzo the first time

for sid in seed_ids[1:2]:

    #Move MUSIC IC to enzo directory
    os.chdir(music_dir)
    for file in glob.glob(r'SEED%s/Grid*' %sid):
        shutil.copy(file, enzo_dir)
    for file in glob.glob(r'SEED%s/Particle*' %sid):
        shutil.copy(file, enzo_dir)
    for file in glob.glob(r'SEED%s/Refinement*' %sid):
        shutil.copy(file, enzo_dir)
    
    #Run enzo
    rw.enzo_run(enzo_dir=enzo_dir,refine=False)

    #Move enzo files to output directories
    process = subprocess.Popen('./clean_intermediate.sh /seeds/SEED%s' %sid, cwd=enzo_dir,
                                shell=True)
    process.wait()

#     #Find halo coordinates
#     enzo_results = '%s/output/seeds/SEED%s' %(enzo_dir,sid)
#     rw.find_halos(enzo_results=enzo_results,halo_coords='%s/coords.v' %enzo_results)

Completed first enzo run.


In [12]:
seed_ids[1:]

['02', '03', '04', '05', '06', '07', '08', '09', '10']

In [3]:
#Find halo coordinates

for sid in seed_ids[:1]:
    
    enzo_results = '%s/output/seeds/SEED%s' %(enzo_dir,sid)
    rw.find_halos(enzo_results=enzo_results,halo_coords='%s/coords.v' %enzo_results)

Creating catalog:  99%|█████████▊| 5202/5276 [00:01<00:00, 3101.80it/s]Found halos.


In [3]:
### Refining initial conditions

for i in seed_ids[:1]:
    seed_idx = i
    refine = True

    enzo_results = '%s/output/seeds/SEED%s' %(enzo_dir, seed_idx)
    halo_coords = '%s/coords.v' %enzo_results
    with open(halo_coords) as c:
        crds = yaml.load(c, Loader=yaml.FullLoader)
    seed = '%s/topostellar-oceans/resources/SEED%s' %(workdir,seed_idx) 

    rw.generate_ic(seed=seed,music_dir=music_dir,
                seed_idx=seed_idx,enzo_dir=enzo_dir,
                music_ex=music_ex,refine=refine, crds=crds)

{'seed': '21188\n', 'seed_id': 'SEED01'}


In [4]:
### Run enzo the second time with varying JLs

for sid in seed_ids[:]:

    #Move MUSIC IC to enzo directory
    os.chdir(music_dir)
    for file in glob.glob(r'SEED%s_initial_centered/Grid*' %sid):
        shutil.copy(file, enzo_dir)
    for file in glob.glob(r'SEED%s_initial_centered/Particle*' %sid):
        shutil.copy(file, enzo_dir)
    for file in glob.glob(r'SEED%s_initial_centered/Refinement*' %sid):
        shutil.copy(file, enzo_dir)

    for i in range(2,8):
        jl = 2**i
        jl_dict = {'jl_val': jl}

        with open(enzo_par_templ, "r") as f:
            contents = f.read()
            f.close()

        with open('%s/parameter_file.txt' %enzo_dir, "a+") as f:
            f.write(contents.format(**jl_dict))
            f.close()

        #Run enzo
        rw.enzo_run(enzo_dir=enzo_dir,refine=True)

        #Move intermediate enzo files to output directories
        process = subprocess.Popen('./clean_intermediate.sh /seeds/SEED%s_JL%s' %(sid,jl),
                                    cwd=enzo_dir, shell=True)
        process.wait()

        open('%s/parameter_file.txt' %enzo_dir, 'w').close()

In [4]:
2**3

8